In [10]:
import numpy as np
from corr_cpl.corr_cpl_functions import luminosity_distance

import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from zusammen.stan_models.stan_model import get_model
from zusammen import AnalysisBuilder, DataSet
from zusammen.spectral_plot import display_posterior_model_counts

In [11]:
N = None
alpha = -1.9 + np.arange(0, 2, 0.2)[0:8]
log_ec = -2 + np.array(range(8))
log_Nrest = 50 + np.array(range(8))
gamma = np.array(range(8))
z = np.array(range(8)) / 4
dl = np.array([luminosity_distance((i)) for i in z])

bool_list = []
data_list = [len(i) for i in [alpha, log_ec, log_Nrest, gamma, z, dl]]
for i in range(len(data_list) - 1):
    bool_list.append(data_list[i] == data_list[i + 1])
if all(bool_list):
    N = data_list[0]

data = dict(
    N=N, alpha=alpha, log_ec=log_ec, log_Nrest=log_Nrest, gamma=gamma, z=z, dl=dl
)


In [12]:
data

{'N': 8,
 'alpha': array([-1.9, -1.7, -1.5, -1.3, -1.1, -0.9, -0.7, -0.5]),
 'log_ec': array([-2, -1,  0,  1,  2,  3,  4,  5]),
 'log_Nrest': array([50, 51, 52, 53, 54, 55, 56, 57]),
 'gamma': array([0, 1, 2, 3, 4, 5, 6, 7]),
 'z': array([0.  , 0.25, 0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75]),
 'dl': array([0.00000000e+00, 4.00772873e+27, 8.99831352e+27, 1.47115405e+28,
        2.09584072e+28, 2.76090277e+28, 3.45745947e+28, 4.17932944e+28])}

In [13]:
mt = get_model("formula_test")
mt.build_model()
mt.model.sample(data=data, fixed_param=True, show_console=True)

08:31:31 - cmdstanpy - INFO - Chain [1] start processing
08:31:31 - cmdstanpy - INFO - Chain [1] done processing


Chain [1] method = sample (Default)
Chain [1] sample
Chain [1] num_samples = 1000 (Default)
Chain [1] num_warmup = 1000 (Default)
Chain [1] save_warmup = 0 (Default)
Chain [1] thin = 1 (Default)
Chain [1] adapt
Chain [1] engaged = 1 (Default)
Chain [1] gamma = 0.050000000000000003 (Default)
Chain [1] delta = 0.80000000000000004 (Default)
Chain [1] kappa = 0.75 (Default)
Chain [1] t0 = 10 (Default)
Chain [1] init_buffer = 75 (Default)
Chain [1] term_buffer = 50 (Default)
Chain [1] window = 25 (Default)
Chain [1] algorithm = fixed_param
Chain [1] num_chains = 1 (Default)
Chain [1] id = 1 (Default)
Chain [1] data
Chain [1] file = /var/folders/jh/4tg_3v1s0_j8p64dw1txwqzr0000gn/T/tmp1jtd5y12/1cjb6152.json
Chain [1] init = 2 (Default)
Chain [1] random
Chain [1] seed = 19687
Chain [1] output
Chain [1] file = /var/folders/jh/4tg_3v1s0_j8p64dw1txwqzr0000gn/T/tmp1jtd5y12/formula_testk81srq_0/formula_test-20221027083131.csv
Chain [1] diagnostic_file =  (Default)
Chain [1] refresh = 100 (Default)


CmdStanMCMC: model=formula_test chains=1['method=sample', 'algorithm=fixed_param']
 csv_files:
	/var/folders/jh/4tg_3v1s0_j8p64dw1txwqzr0000gn/T/tmp1jtd5y12/formula_testk81srq_0/formula_test-20221027083131.csv
 output_files:
	/var/folders/jh/4tg_3v1s0_j8p64dw1txwqzr0000gn/T/tmp1jtd5y12/formula_testk81srq_0/formula_test-20221027083131_0-stdout.txt

In [14]:
def epeak_func(alpha, ec):
    return (2 + alpha) * ec

def energy_flux_func(Nrest, gamma, z, dl, epeak):
    return Nrest / (4 * np.pi * dl**2) * ((1 + z) * epeak / 100)**gamma

In [15]:
epeak_py = [epeak_func(i, j) for i, j in zip(alpha, 10.**log_ec)]
energy_flux_py = [
    energy_flux_func(i, j, h, k, l)
    for i, j, h, k, l in zip(10.**log_Nrest, gamma, z, dl, epeak_py)
]
np.log10(epeak_py),epeak_py, energy_flux_py

(array([-3.        , -1.52287875, -0.30103   ,  0.84509804,  1.95424251,
         3.04139269,  4.11394335,  5.17609126]),
 [0.0010000000000000009,
  0.030000000000000006,
  0.5,
  7.000000000000002,
  90.00000000000001,
  1100.0,
  13000.000000000002,
  150000.00000000003],
 [inf,
  1.8579104055124015e-09,
  5.528284965255136e-10,
  6.75899460249164e-08,
  0.0019017945919667131,
  9695.359700313023,
  7844695793278.324,
  9.258584487990407e+23])

In [16]:
from corr_cpl.corr_cpl_functions import golenetskii_corr
energy_flux_zusammen = [
    golenetskii_corr(i, j, h, k)
    for i, j, h, k in zip(epeak_py, 10.**log_Nrest, gamma, z)
]
energy_flux_zusammen

[inf,
 1.8579104055123977e-09,
 5.528284965255118e-10,
 6.758994602491648e-08,
 0.0019017945919666936,
 9695.359700313085,
 7844695793278.561,
 9.258584487990317e+23]

Seems to match